In [ ]:
import sys
import xarray as xr
import numpy as np
import datetime 
import time
from datetime import datetime 
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
#import sklearn
from sklearn.metrics import median_absolute_error, mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression, RANSACRegressor, HuberRegressor


import pyOptimalEstimation as pyOE

import cartopy.crs as ccrs
from cmocean import cm as cmo

import matplotlib.ticker as mticker

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
sys.path.append('support')

import supporting_routines_m 

import os

rttov_installdir = '/home/mario/myLibs/rrtov13/rttov130'

sys.path.append(rttov_installdir+'/wrapper')
import pyrttov

In [ ]:
aprio_dir = '/home/mario/Data/RadEst_crossVal_data/crossVal_aprioriData/'
prior_all = supporting_routines_m.concatDiverse(aprio_dir)

In [ ]:
# Number of levels for the profiles in the priors

nlevels = len(prior_all['height'])
nprofiles = 1

In [ ]:
# Channels corrispondence:

# from: D. B. Kunkee et al, "Design and Evaluation of the First 
# Special Sensor Microwave Imager/Sounder", IEEE Trans. Geosc. Rem. Sens. 
# Vol. 46, NO. 4, April 2008.
# Table 1
# Ch. 1 - 50.3 GHz Hpol (check note on pol at source for this channel) 
# Ch. 12 - 19.35 GHz Hpol
# Ch. 13 - 19.35 GHz Vpol
# Ch. 14 - 22.235 GHz Vpol
# Ch. 15 - 37 GHz Hpol
# Ch. 16 - 37 GHz Vpol

ssmiRttov = pyrttov.Rttov()

# SSMIS:
nchan_ssmi = 14
chan_list_ssmi = (1,2,3,4,5,6,7,12,13,14,15,16,23,24) #(1,12,13,14,15,16) #  #
ssmiRttov.FileCoef = '{}/{}'.format(rttov_installdir,
                                    "rtcoef_rttov13/rttov7pred54L/rtcoef_dmsp_18_ssmis.dat")

# WindSat:
#nchan_ssmi = 16
#chan_list_ssmi = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) #(1,2,3,4,7,8,11,12,13,14) #(1,12,13,14,15,16) #  #
#ssmiRttov.FileCoef = '{}/{}'.format(rttov_installdir,
#                                    "rtcoef_rttov12/rttov7pred54L/rtcoef_coriolis_1_windsat.dat")

ssmiRttov.Options.AddInterp = True
ssmiRttov.Options.CO2Data = False
ssmiRttov.Options.VerboseWrapper = True
ssmiRttov.Options.DoCheckinput = False
ssmiRttov.Options.UseQ2m = True
ssmiRttov.Options.ApplyRegLimits = True
ssmiRttov.Options.Verbose = False
ssmiRttov.Options.FastemVersion = 6 

# Load the instruments: for HIRS and MHS do not supply a channel list and
# so read all channels
try:
    ssmiRttov.loadInst(chan_list_ssmi)
except pyrttov.RttovError as e:
    sys.stderr.write("Error loading instrument(s): {!s}".format(e))
    sys.exit(1)

# Load Atlases: (if any)    
ssmiRttov.SurfEmisRefl = np.zeros((4,nprofiles,nchan_ssmi), dtype=np.float64) # RTTOVv12 used (2,nprof,nchan)

# Definition of the observation variables
y_vars = np.array(chan_list_ssmi)

In [ ]:
# ONLY FOR MULTIPLE PROFILES CALL TESTING

ssmiRttovM = pyrttov.Rttov()

# SSMIS:
nchan_ssmiM = 14
chan_list_ssmiM = (1,2,3,4,5,6,7,12,13,14,15,16,23,24) #(1,12,13,14,15,16) #  #
ssmiRttovM.FileCoef = '{}/{}'.format(rttov_installdir,
                                    "rtcoef_rttov13/rttov7pred54L/rtcoef_dmsp_18_ssmis.dat")

ssmiRttovM.Options.AddInterp = True
ssmiRttovM.Options.CO2Data = False
ssmiRttovM.Options.VerboseWrapper = True
ssmiRttovM.Options.DoCheckinput = False
ssmiRttovM.Options.UseQ2m = True
ssmiRttovM.Options.ApplyRegLimits = True
ssmiRttovM.Options.Verbose = False
ssmiRttovM.Options.FastemVersion = 6 

# Load the instruments: for HIRS and MHS do not supply a channel list and
# so read all channels
try:
    ssmiRttovM.loadInst(chan_list_ssmiM)
except pyrttov.RttovError as e:
    sys.stderr.write("Error loading instrument(s): {!s}".format(e))
    sys.exit(1)

# Load Atlases: (if any)    
ssmiRttovM.SurfEmisRefl = np.zeros((4,279,nchan_ssmi), dtype=np.float64) # RTTOVv12 used (2,nprof,nchan)



In [ ]:
# Values for the noise are obtained from HOAPS R matrix (Table 2, p20, HOAPS 4.0 ATBD)

#y_noise = pd.Series(
#    [
#        2.44948974, 1.54919338, 1.341640787, 1.341640787, 1.949358869, 1.341640787
#    ],
#    index=y_vars
#)

#S_y = pd.DataFrame(
#    np.diag(y_noise.values**2),
#    index=y_vars,
#    columns=y_vars,
#)
#

# JUST FOR TESTING THE EFFECT ON SLOPE VIAS IN CROSS-VALIDATION (DON'T USE IN REAL APPLICATION)
# Channels 1-7, 12-16, 23, 24 (MODIFIED values from Deblonde-English 2003) (sigma or std)
#y_noise = pd.Series(
#    [
#        1.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 2.0, 1.07, 1.24, 2.5, 1.14, 0.46, 0.47
#   ],
#    index=y_vars
#)
# JUST FOR TESTING THE EFFECT ON SLOPE VIAS IN CROSS-VALIDATION (DON'T USE IN REAL APPLICATION)


# FOR TEST USING NEDT VALUES FROM SSMIS ONLY (From Kunkee 2008)
# Channels 1-7, 12-16, 23, 24 (values in K)
#y_noise = pd.Series(
#    [
#        0.34, 0.32, 0.33, 0.33, 0.34, 0.41, 0.40, 0.33, 0.31, 0.43, 0.25, 0.20, 0.8, 0.9 
#   ],
#    index=y_vars
#)
# FOR TEST USING NEDT VALUES FROM SSMIS ONLY (From Kunkee 2008)


# Channels 1-7, 12-16, 23, 24 (values from Deblonde-English 2003) (sigma or std)
y_noise = pd.Series(
    [
        1.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 2.4, 1.27, 1.44, 3.0, 1.34, 0.46, 0.47
   ],
    index=y_vars
)

# Channels 1-7, 13,14,16, 23, 24 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pd.Series(
#    [
#        1.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 1.27, 1.44, 1.34, 0.46, 0.47
#    ],
#    index=y_vars
#)

# Channels 1,12-16 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pd.Series(
#    [
#        1.5, 2.4, 1.27, 1.44, 3.0, 1.34
#    ],
#    index=y_vars
#)

# Channels 2-7, 23, 24 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pd.Series(
#    [
#        0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.46, 0.47
#    ],
#    index=y_vars
#)


# Channels 1-16 WindSat (from Table 2 in M. Bettenhausen et al, 
# "A Nonlinear Optimization Algorithm for WindSat Wind Vector Retrievals", 
# IEEE Trans. Geo. Rem. Sen., Vol 44, No. 3, March 2006):

# Take "Polarimetric" channels away:
#y_noise = pd.Series(
#    [
#        0.60, 0.78, 0.69, 0.99, 1.02, 2.02, 1.38,
#        2.51, 1.76, 3.65
#    ],
#    index=y_vars
#)

#y_noise = pd.Series(
#    [
#        0.60, 0.78, 0.69, 0.99, 0.26, 0.09, 1.02, 2.02, 0.28, 0.12, 1.38,
#        2.51, 1.76, 3.65, 0.25, 0.09
#    ],
#    index=y_vars
#)

# Variance values > std**2
S_y = pd.DataFrame(
    np.diag(y_noise.values**2),
    index=y_vars,
    columns=y_vars,
)

In [ ]:
# Definition of the indices of the "local prior":
# we will call local prior to the prior built
# using a geographical area "nearby" the observation
# location (i.e. a bounding box around the observation location). 
# All the atmospheric states within such a box 
# Define our a-priori knowledge (in the spatial domain)

lat_min = -90.0
lat_max = -40.0
long_min = -180.0
long_max = 180.0


lat_long_mask = np.array([(prior_all['lat'].values[:]>lat_min)&(prior_all['lat'].values[:]<lat_max)&
                 ((prior_all['long'].values[:]>long_min)&(prior_all['long'].values[:]<long_max))])
lat_long_mask=lat_long_mask.reshape(-1)
#lat_long_mask = ~lat_long_mask


In [ ]:
# Creating a time index for the local prior:
times1 = prior_all['Time'].values[lat_long_mask]

time_index = np.zeros(len(times1),dtype='datetime64[s]')
for i in range(len(times1)):
    time_index[i] = supporting_routines_m.timestamp2datetime(times1[i])
print(len(times1))    

In [ ]:
# Creating lat, long vectors for auxiliary plots e.g. cartopy

lat = prior_all['lat'].values[lat_long_mask]
long = prior_all['long'].values[lat_long_mask]


In [ ]:
# Creating a "local" prior: Subset of "prior_all" 
# that contains only the selected lat,long area.

prior_local = supporting_routines_m.maskPrior(prior_all, time_index, lat_long_mask)


In [ ]:
# Transform the humidity profiles to a logarithmic scale
# in order to improve their normality (see probability plots)

# IMPORTANT NOTE:
# RTTOV ACCEPTS ONLY kg / kg (LINEAR SCALE)
# THERE IS A LOG2LIN CONVERSION IN THE DEFINITION
# OF THE FORWARD OPERATOR; BUT INSIDE THE OPTIMAL
# ESTIMATION, THE HUMIDITY IS IN LOG SCALE

prior_local['Humidity'] = np.log10(prior_local['Humidity'])

In [ ]:
indexTotal = np.arange(len(prior_local.time)) # all time indices in prior_local [0,1,...16000,..] 

# Create copy of the indices array
kk = np.copy(indexTotal)

# Shuffle the indices array randomly
np.random.shuffle(kk)

# Split the array in K folds (for K-fold cross validation)
K = 10
hh = np.array_split(kk,K)

rmse = 0.0
mae = 0.0 #np.empty(0)
r2 = 0.0 #np.empty(0)
count = np.empty(0,dtype=int)
wind_opt = np.empty(0)
wind_true = np.empty(0)
wind_opt_err = np.empty(0)

KK = 0

for indexTest in hh: 
    
    KK += 1
# How many "situations" do we want to test 
#(these will be split apart from the a-priori dataset).
# Situation means a (Lat, Long, time) combination:
# i.e. 1 Situation is one datapoint from the prior_local dataset
    nTestSamples = len(indexTest) 

# The actual indices from prior_local  that will be used as testing points: 
# i.e. they are taken out of the a-priori data
#indexTest = np.random.choice(indexTotal,nTestSamples) # This is used only if a Monte Carlo type of cross validation is done

# The test indices are deleted from the a-priori (i.e. training) set
    indexTrain = np.delete(indexTotal,indexTest)

# Split available diverse datasets into:
# - 5 profiles to generate synthetic data (profiles)
# - Rest of the profiles to generate the prior 
# The 5 profiles for synthetic data are EXCLUDED from
# the rest of the datasets (e.g. from the prior)

    profiles = prior_local.isel(time=indexTest)
    prior_local_1 = prior_local.isel(time=indexTrain)
    
    nLev = len(prior_local_1.height)
    # Splitting the priors per season:
    priors, pressure, seasons, months, h_season = supporting_routines_m.priors2seasons(prior_local_1) 
    # Converting priors per season to pandas dataframes:
    #prior_xa, prior_wind = supporting_routines_m.priors2pandas(priors, 
    #                                                            wind_components = True, h_season = h_season)
    
    flavor = 2 # check "priors2Pandas"
    prior_xa, prior_b = supporting_routines_m.priors2Pandas(priors, 
                                                                flavor = flavor, h_season = h_season)    
    x_cov, x_mean = supporting_routines_m.meanCov(prior_xa, seasons)

# Definition of the state variables (names of the variables)
    x_vars = x_mean.state.values

# Assert invertibility of the convariance matrices:
    for season in x_cov.season:
        assert np.linalg.matrix_rank(
            x_cov.sel(season=season).to_pandas()) ==  x_cov.shape[-1]
        
# Create x_truths: Pandas version of the profiles array (Containing only state variables!)
    x_truths = supporting_routines_m.createTrueState(profiles, flavor=flavor)
# Gather parameters for RTTOV from a-priori knowledge
    
    #prior_b = supporting_routines_m.b_fromPrior(priors)
    
# FOR RTTOV PARAMETERS
# We use pandas tooling (cov, mean) to compute the mean
# and covariance per season:  
    b_cov, b_mean = supporting_routines_m.meanCov(prior_b, seasons)
    b_vars = b_mean.state.values

# Assert invertibility of the convariance matrices:
    for season in x_cov.season:
        assert np.linalg.matrix_rank(
            b_cov.sel(season=season).to_pandas()) ==  b_cov.shape[-1]   
    
    #w_op = np.zeros(len(indexTest))
    #w_op_err = np.zeros(len(indexTest))
    #w_a = np.zeros(len(indexTest))
    #w_a_err = np.zeros(len(indexTest))
    #w_truth = np.zeros(len(indexTest))
    #chiSquareTest = np.array(len(indexTest), dtype=bool)
    #linearityTest = np.array(len(indexTest), dtype=bool)
    w_op2 = np.empty(0)
    w_op_err2 = np.empty(0)
    w_truth2 = np.empty(0)
    u_op2 = np.empty(0)
    u_op_err2 = np.empty(0)
    u_truth2 = np.empty(0)
    v_op2 = np.empty(0)
    v_op_err2 = np.empty(0)
    v_truth2 = np.empty(0)    
    lat_local = np.empty(0)
    long_local = np.empty(0)
    lat_skip = np.empty(0)
    long_skip = np.empty(0)
    seasonal = np.empty(0,dtype=object)
        
    count_breaks = 0
    count_nc = 0
    for ind in np.arange(len(indexTest)):

        prof = profiles.time.values[ind] 
        seasonIndex = int(np.where(np.isin(months,profiles['time.month'].values[ind]))[0])
        season_prof = seasons[seasonIndex]

# Profile used for generating synthetic observation:
# Contains ONLY the to-be-retrieved parameters: 
# temperature, humidity, 10m surface windspeed
        x_truth = x_truths.iloc[ind]

        Pressure = pressure.loc[season_prof,:].values[:] 

# Other parameters needed for the radiative transfer model (RTTOV)
        lat1 = np.float64(profiles.lat.values[ind])
        long1 = np.float64(profiles.long.values[ind])
        datetime_obs = prof
        zenithAngle = 53.0
        salinity = 35

# Create instance of Profiles class; 
# it's a container of the input atmospheric state that RTTOV will simulate

        myProfiles = pyrttov.Profiles(nprofiles, nlevels)

# Initialize multiple profiles for using a single call to RTTOV
# This is to be passed to the Jacobian function inside pyOpEst:
# The Jacobian is needed per parameter (len(x_truth.index))
        nprofiles_test = len(x_truth.index) # total number of parameters (x and b)
        myProfilesM = pyrttov.Profiles(nprofiles_test, nlevels) #  #Testing!

# forward_b_init fills "myProfiles" with the "fixed" parameters for the RTTOV simulation.
# The forward model F(x,b), RTTOV in our case, has two "parameters": x and b
# x is the state vector that is being retrieved (as such it is allowed to change during the retrieval)
# b contains all other parameters that are fixed during the retrieval (everything else that is not being retrieved)

        forward_b_init(Pressure, salinity, 
              lat1, long1, datetime_obs, zenithAngle, myProfiles)

    
        press2 = np.zeros((nlevels,nprofiles_test))
        x_truth_aux_data = np.zeros((nprofiles_test,nprofiles_test), dtype=np.float64) 
        columns_xtruth = []
        for jj in np.arange(nprofiles_test):
            press2[:,jj] = pressure.loc[season_prof,:].to_numpy()
            x_truth_aux_data[:,jj] = x_truth.to_numpy() 
            columns_xtruth.append('profile_'+str(jj))

        x_truth_aux = pd.DataFrame(x_truth_aux_data,
            columns=columns_xtruth, index=x_truth.index, dtype=np.float64)
        #print(x_truth_aux.head())
        
# Initialize profile datastructure for use in Jacobian computation:        
        forward_b_init(press2, salinity, 
              lat1, long1, datetime_obs, zenithAngle, myProfilesM)    
        
# Define a-priori information: mean and covariance matrix
# for a given season (DJF, MAM, JJA, SON)

        x_a = x_mean.sel(season=season_prof).to_pandas()[x_vars]
        S_a = x_cov.sel(season=season_prof).to_pandas().loc[x_vars, x_vars]
        
        # Vector of parameters and its covariance:
        b_p = b_mean.sel(season=season_prof).to_pandas()[b_vars]
        S_b = b_cov.sel(season=season_prof).to_pandas().loc[b_vars, b_vars]
            
# Define dictionary of parameters for the forward model:

        #forwardKwArgs = dict(
        #    myProfiles_a = myProfiles, 
        #    ssmiRttov_a = ssmiRttov
        #)
        forwardKwArgs = {"myProfiles_a" : myProfiles, 
                         "ssmiRttov_a" : ssmiRttov}
                
# Test for multiple profiles   
        #forwardKwArgsM = dict(
        #    myProfiles_a = myProfilesM, 
        #    ssmiRttov_a = ssmiRttovM
        #)  
        forwardKwArgsM = {"myProfiles_a" : myProfilesM, 
                         "ssmiRttov_a" : ssmiRttovM}    
       
        
# Call the forward model (i.e. creating synthetic obs.)

        y_obs = forwardRT(x_truth, **forwardKwArgs)
        y_obs = pd.Series(y_obs, index=y_vars)
        assert np.all(np.isfinite(y_obs))
        
        #y_K = KRT(x_truth, **forwardKwArgs)  # only for testing runK model from RTTOV, experimental!

        #print('y_obs done') 
# Create optimalEstimation instance:

        oe_ref = pyOE.optimalEstimation( # oe_1 if windDisambiguation used
            x_vars, # state variable names
            x_a,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            forwardKwArgsM=forwardKwArgsM, # additonal function arguments for jacobian tests
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
        )

        #oe_1.doRetrieval()
        oe_ref.doRetrieval()
        
        #time.sleep(3600)
        
        if not oe_ref.converged :  # oe_1 if windDisambiguation used
            count_nc += 1
            print('NO CONVERGENCE points:')
            print(count_nc)
            lat_skip = np.append(lat_skip,lat1)
            long_skip = np.append(long_skip,long1)
            continue

        #oe_ref = windDisambiguation(oe_1,forwardRT,forwardKwArgs)  
        
# Not Including S_b, b_p:            
        #_, _, w_op1 = supporting_routines_m.splitX(oe_ref.x_op)
        #_, _, w_op_err1 = supporting_routines_m.splitX(oe_ref.x_op_err)
        ##_, _, w_a[ind] = supporting_routines_m.splitX(oe_ref.x_a)
        ##_, _, w_a_err[ind] = supporting_routines_m.splitX(oe_ref.x_a_err)
        #_, _, w_truth1 = supporting_routines_m.splitX(oe_ref.x_truth)
        
    # U & V:
        #_, _, u_op1, v_op1 = supporting_routines_m.splitXUV(oe_ref.x_op)
        #_, _, u_op_err1, v_op_err1 = supporting_routines_m.splitXUV(oe_ref.x_op_err)
        ##_, _, u_a1, v_a1 = supporting_routines_m.splitXUV(oe_ref.x_a)
        ##_, _, u_a_err1, v_a_err1 = supporting_routines_m.splitXUV(oe_ref.x_a)
        #_, _, u_truth1, v_truth1 = supporting_routines_m.splitXUV(oe_ref.x_truth) 
        #u_op2 = np.append(u_op2,u_op1)
        #u_op_err2 = np.append(u_op_err2,u_op_err1)
        #u_truth2 = np.append(u_truth2,u_truth1)
        ## V component
        #v_op2 = np.append(v_op2,v_op1)
        #v_op_err2 = np.append(v_op_err2,v_op_err1)
        #v_truth2 = np.append(v_truth2,v_truth1)          
        ## Compute magnitude using coordinates U and V
        ## w_op1 = np.sqrt( u_op1**2 + v_op1**2 )
        #w_op1, w_op_err1 = supporting_routines_m.UV2Wvar(oe_ref.S_op.loc['00010_nu','00010_nu'],
        #                                          oe_ref.S_op.loc['00010_nv','00010_nv'],
        #                                          oe_ref.S_op.loc['00010_nu','00010_nv'],
        #                                          u_op1,v_op1)
        #w_truth1 = np.sqrt( u_truth1**2 + v_truth1**2 )        
        
# Including S_b, b_p:
       # _,_, w_op1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_op)
       # _,_, w_op_err1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_op_err)
       ##_,_, w_a,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_a)
       ##_,_, w_a_err,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_a_err)
       # _,_, w_truth1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_truth)
        
    # U & V:
        _,_,u_op1, v_op1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op)
        _,_,u_op_err1, v_op_err1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op_err)
        #_,_,u_a1, v_a1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_a)
        #_,_,u_a_err1, v_a_err1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_a_err)
        _,_,u_truth1, v_truth1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_truth)
        u_op2 = np.append(u_op2,u_op1)
        u_op_err2 = np.append(u_op_err2,u_op_err1)
        u_truth2 = np.append(u_truth2,u_truth1)
        v_op2 = np.append(v_op2,v_op1)
        v_op_err2 = np.append(v_op_err2,v_op_err1)
        v_truth2 = np.append(v_truth2,v_truth1)          
    # Compute magnitude using coordinates U and V
        w_op1, w_op_err1 = supporting_routines_m.UV2Wvar(oe_ref.S_op.loc['00010_nu','00010_nu'],
                                                  oe_ref.S_op.loc['00010_nv','00010_nv'],
                                                  oe_ref.S_op.loc['00010_nu','00010_nv'],
                                                  u_op1,v_op1)        
        w_truth1 = np.sqrt( u_truth1**2 + v_truth1**2 )   
        
        # For wind components we compare absolute value only
        # SSMIS cannot provide polarimetric measurements
        # so comparing angular information is not a proper metric
        # U component 
        

        w_op2 = np.append(w_op2,w_op1)
        w_op_err2 = np.append(w_op_err2,w_op_err1)
        w_truth2 = np.append(w_truth2,w_truth1)
        
        
        
        seasonal = np.append(seasonal,season_prof)
        
        lat_local = np.append(lat_local,lat1)
        long_local = np.append(long_local,long1)
        
        print('index:')
        print(ind)
        
    
    plotInMap(lat_local, long_local, lat, long, KK, lat_skip, long_skip)
    perSeasonPlots(w_truth2, w_op2, seasonal, KK, windType = 'W10m')
    perSeasonPlots(u_truth2, u_op2, seasonal, KK, windType = 'U10m')
    perSeasonPlots(v_truth2, v_op2, seasonal, KK, windType = 'V10m')
    
    rmse_local, mae_local, r2_local = regressPlots(w_truth2, w_op2, KK, windType = 'W10m')   
    _, _, _ = regressPlots(u_truth2, u_op2, KK, windType = 'U10m')
    _, _, _ = regressPlots(v_truth2, v_op2, KK, windType = 'V10m')
    
    rmse += rmse_local
    mae += mae_local
    r2 += r2_local
    
    #skipRatio = count_nc/(count_nc+len(w_op2)) # ratio of (skipped / total) points in the test dataset
    
    
    
    #if KK == 1: break

rmse = rmse / KK
mae = mae / KK
r2 = r2 / KK


In [ ]:
oe_ref.S_op.loc[['00010_nu','00010_nv'],['00010_nu','00010_nv']]
Uu2 = oe_ref.S_op.loc['00010_nu','00010_nu']  # variance U
Uv2 = oe_ref.S_op.loc['00010_nv','00010_nv']  # variance V 
Uuv = oe_ref.S_op.loc['00010_nu','00010_nv']  # covariance UV

u = np.float64(u_op1.values)
v = np.float64(v_op1.values)
w = np.sqrt(u**2+v**2)

bruteUncertainty = np.sqrt(np.float64(u_op_err1.values)**2+np.float64(v_op_err1.values)**2)

combinedUncertainty = np.sqrt((1/(w**2))*(Uu2*(u)**2+Uv2*(v)**2+2*u*v*Uuv))

print(oe_ref.d_i2)

In [ ]:
def windDisambiguation(oe_ref,forwardRT,forwardKwArgs):
    # Procedure proposed in:
    # M. Bettenhausen, et al, "A Nonlinear Optimization Algorithm 
    # for WindSat Wind Vector Retrievals", IEEE Trans. Geo. Rem. Sens.,
    # Vol. 44, No. 3, March 2006
    
    # First retrieved wind
    #_, _, u_op, v_op = supporting_routines_m.splitXUV(oe_ref.x_op)
    _,_,u_op, v_op,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op)
    
    # Most of the retrieval parameters are the same as for the original retrieval:
    x_vars = oe_ref.x_vars 
    y_vars = oe_ref.y_vars 
    y_obs = oe_ref.y_obs 
    S_a = oe_ref.S_a
    S_y = oe_ref.S_y
    
    S_b = oe_ref.S_b
    b_vars = oe_ref.b_vars
    b_p = oe_ref.b_p
    
    #forwardRT = oe_ref.forwardRT
    #forwardKwArgs = oe_ref.forwardKwArgs
    x_truth = oe_ref.x_truth 
    
    mini = 100000
    
    rot = np.array([[0,-1],[1,0]]) # Rotation matrix (90 deg rotation)
    # 4 ambiguities (1 per quadrant):
    aprio1 = np.array([[np.float64(u_op.values)],[np.float64(v_op.values)]])   
    aprio2 = np.dot(rot,aprio1)
    aprio3 = np.dot(rot,aprio2)
    aprio4 = np.dot(rot,aprio3)
    
    # Re-use the first retrieved atmospheric state as the new a-priori dataset
    x_a1 = oe_ref.x_op.copy()
    x_a1['00010_nu'] = aprio1[0,0] # a-priori wind 1: as retrieved
    x_a1['00010_nv'] = aprio1[1,0]
    oe_ref1 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a1,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref1.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario    
    numero = np.dot(np.transpose(y_obs-oe_ref1.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref1.S_y),
                           (y_obs-oe_ref1.y_i[-1])))
    #if oe_ref1.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref1.d_i2[-1]
        oe_return = oe_ref1
        index = 1
        
    #_, _,u1, v1 = supporting_routines_m.splitXUV(oe_ref1.x_op)
# 2nd vector
    x_a2 = oe_ref.x_op.copy()
    x_a2['00010_nu'] = aprio2[0,0]
    x_a2['00010_nv'] = aprio2[1,0]
    oe_ref2 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a2,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref2.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario
    numero = np.dot(np.transpose(y_obs-oe_ref2.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref2.S_y),
                           (y_obs-oe_ref2.y_i[-1])))    
    #if oe_ref2.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref2.d_i2[-1]
        oe_return = oe_ref2
        index = 2
        
    #_, _,u2, v2 = supporting_routines_m.splitXUV(oe_ref2.x_op)
# 3rd vector
    x_a3= oe_ref.x_op.copy()
    x_a3['00010_nu'] = aprio3[0,0]
    x_a3['00010_nv'] = aprio3[1,0]
    oe_ref3 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a3,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref3.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario    
    numero = np.dot(np.transpose(y_obs-oe_ref3.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref3.S_y),
                           (y_obs-oe_ref3.y_i[-1])))     
    
    #if oe_ref3.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref3.d_i2[-1]
        oe_return = oe_ref3
        index = 3
        
    #_, _,u3, v3 = supporting_routines_m.splitXUV(oe_ref3.x_op)
# 4th vector
    x_a4 = oe_ref.x_op.copy()
    x_a4['00010_nu'] = aprio4[0,0]
    x_a4['00010_nv'] = aprio4[1,0]
    oe_ref4 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a4,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref4.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario
    numero = np.dot(np.transpose(y_obs-oe_ref4.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref4.S_y),
                           (y_obs-oe_ref4.y_i[-1])))     
    
    #if oe_ref4.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref4.d_i2[-1]
        oe_return = oe_ref4
        index = 4
        
    #_, _,u4, v4 = supporting_routines_m.splitXUV(oe_ref4.x_op)
    print(index)

    return oe_return

In [ ]:
def forwardRT(X, myProfiles_a, ssmiRttov_a):
    
    # TODO: Add assertions, tests *** mario

    # X contains T, Q and W10, lets split the vector
    #temperature, humidity, wind10m = supporting_routines_m.splitX(X)
    
    # if wind speed in components:
    #temperature, humidity, u10m, v10m = supporting_routines_m.splitXUV(X)
    temperature, humidity, u10m, v10m, bp2m, bt2m, btsk \
    = supporting_routines_m.splitX_all(X)
    #temperature, humidity, wind10m, bp2m, bt2m, btsk = supporting_routines_m.splitXW_all(X)
    
    
    
    # humdity is in log10 scale, convert to linear in kg/kg
    humidity = (10**humidity) / 1000.
    # or abs_humidity? *** note mario

    myProfiles_a.T = reshape4profiles(temperature.to_numpy(dtype=np.float64))  
    myProfiles_a.Q = reshape4profiles(humidity.to_numpy(dtype=np.float64))  

    myProfiles_a.S2m[:,0] = reshape4profiles(
        bp2m.to_numpy(dtype=np.float64)).flatten() # surface pressure
    myProfiles_a.S2m[:,1] = reshape4profiles(
        bt2m.to_numpy(dtype=np.float64)).flatten()  # 2m temperature
    myProfiles_a.Skin[:,0] = reshape4profiles(
        btsk.to_numpy(dtype=np.float64)).flatten() 
    
    # if wind in components:
    myProfiles_a.S2m[:,3] = reshape4profiles(
        u10m.to_numpy(dtype=np.float64)).flatten()  #  10m windspeed, u component
    myProfiles_a.S2m[:,4] = reshape4profiles(
        v10m.to_numpy(dtype=np.float64)).flatten()  #  10m windspeed, v component  
    
   
    ssmiRttov_a.Profiles = myProfiles_a
    
    ssmiRttov_a.SurfEmisRefl[:,:,:] = -1. # need to "reset" to -1 every time RTTOV is called; 
    # -1 indicates to RTTOV to use internal values for surface emissivity.

    try:
        ssmiRttov_a.runDirect()
    except pyrttov.RttovError as e:
        sys.stderr.write("Error running RTTOV direct model: {!s}".format(e))
        sys.exit(1)    
        
    #print(ssmiRttov.BtRefl[:, :].shape)
    #print(ssmiRttov.BtRefl[:, :])
    
    if(ssmiRttov_a.BtRefl[:, :].shape[0]==1):
        TB = ssmiRttov_a.BtRefl[0, :].T
    else:
        TB = ssmiRttov_a.BtRefl[:, :].T
    
    return TB




def reshape4profiles(profiles):
    # "profiles" is a numpy array
    # "profiles" can contain 1 or more profiles
    # "profiles" has dimensions (nlevels, nprofiles)
    # "outProfiles" has dimensions (nprofiles,nlevels) (as needed in RTTOV)
    
    if (len(profiles.shape)==1):
        outProfiles = profiles.reshape(1,profiles.shape[0]).copy()
    else:
        outProfiles = profiles.T.copy() #profiles.reshape(profiles.shape[1]
            #                          ,profiles.shape[0]) 
    return outProfiles    

def expand2nprofiles(n, nprof):
    # Transform 1D array to a [nprof, nlevels] array
    outp = np.empty((nprof, len(n)), dtype=n.dtype)
    for i in range(nprof):
        outp[i, :] = n[:]
    return outp


def forward_b_init(pressure, salinity, lat, long, datetime_obs64, 
                   zenithAngle, myProfiles):
    
    if (len(pressure.shape)==1):
        nprofiles = 1
    else:
        nprofiles =  pressure.shape[1]
    
    # The rest of the code uses datetime64 format (numpy), but I have to pass the obs date as integers to RTTOV
    datetime_obs = supporting_routines_m.datetime64_to_datetime(datetime_obs64)
    
    s2m = np.zeros((nprofiles,6), dtype=np.float64) # s2m has 6 elements (docs RTTOV)
    
    angles = np.zeros((nprofiles,4), dtype=np.float64) # angles has 4 elements (docs RTTOV)
    angles[:,0] = zenithAngle
    
    
    # for RTTOV 13 skin is 9 elements long:
    skin = np.zeros((nprofiles,9), dtype=np.float64) # skin has 9 elements (docs RTTOV)
    skin[:,1] = salinity
        
    surftype = np.zeros((nprofiles,2), dtype=np.int32) # surftype has 2 elements (docs RTTOV)
    surftype[:,:] = 1 # [sea, ocean] Harcoded for now, TODO *** mario
    
    
    surfgeom = np.zeros((nprofiles,3), dtype=np.float64) # surfgeom has 3 elements (docs RTTOV)
    surfgeom[:,0] = lat
    surfgeom[:,1] = long
    # surfgeom[:,2]=0 # elevation harcoded to 0 for now, TODO *** mario
    
    date_times = np.zeros((nprofiles,6), dtype=np.int32) # date_times has 6 elements (docs RTTOV)
    date_times[:,0] = datetime_obs.year
    date_times[:,1] = datetime_obs.month
    date_times[:,2] = datetime_obs.day
    date_times[:,3] = datetime_obs.hour
    date_times[:,4] = datetime_obs.minute
    date_times[:,5] = datetime_obs.second
    
    
    myProfiles.GasUnits = 1  # kg/kg (see RTTOV doc. for other options) # Harcoded for now, TODO *** mario
    myProfiles.P = reshape4profiles(pressure) 
    myProfiles.S2m = s2m
    myProfiles.Angles = angles
    myProfiles.Skin = skin
    myProfiles.SurfType = surftype
    myProfiles.SurfGeom = surfgeom
    myProfiles.DateTimes = date_times
    
    #return myProfiles

In [ ]:
def plotInMap(lat2, long2, lat, long, indice, lat_s = None, long_s = None):
    
    #lat2 = profiles.lat.values
    #long2 = profiles.long.values

    plt.figure(figsize=(13,6.2))
    
    ax = plt.subplot(111, projection=ccrs.PlateCarree())

    ax.scatter(long,\
                   lat,marker='.',color='red')
    ax.scatter(long2,\
                   lat2,marker='o',color='blue')
    if (lat_s is not None)and(long_s is not None):
        ax.scatter(long_s,\
                   lat_s,marker='o',color='yellow')
                
    ax.coastlines();

    ax.grid(True)

    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')

    ax.stock_img();

    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, 
                      color='black', alpha=0.5, linestyle='--', draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.ylabels_right=True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30, 60])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    plt.savefig('Loc_Fold_'+str(indice)+'.png', dpi=300)

def perSeasonPlots(truth,optimized,seasonal, indice, windType = 'W10m'):
    
    colors = {'DJF':'blue', 'MAM':'green', 'JJA':'orange', 'SON':'purple'}
    
    df = pd.DataFrame(dict(Truth=truth, Optimized=optimized, Season = seasonal))
    
    seasonGroup = df.groupby('Season')

    for name, group in seasonGroup:
        #print name
        #print group 
        
        w_truth = group['Truth'].values[:]
        w_opt = group['Optimized'].values[:]

        error_local = w_truth - w_opt
        mean_error_local = np.mean( error_local )
        median_error_local = np.median( error_local )
        multiplicativeBias = np.mean(w_truth) / np.mean(w_opt)

        rmse_local = np.sqrt(mean_squared_error(w_truth, w_opt)) 
        mae_local = median_absolute_error(w_truth, w_opt)
        r2_local = r2_score(w_truth, w_opt)

        
        wt = np.reshape( w_truth, (len(w_truth),1) ) # for using with .fit below
        
        # Ordinary least squares:
        regr = LinearRegression()
    
        # Robust fit with Ransac algo. (sklearn lib.)
        ransac = RANSACRegressor()
        
        # HuberRegressor
        #huber = HuberRegressor()

        # Train the model using the training sets
        
            # OLS fit:
        regr.fit(wt, w_opt)
        # Ransac fit:
        ransac.fit(wt, w_opt)
        # Huber fit
        #huber.fit(wt, w_opt)
        
        # mask inlier-outlier from Ransac:
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)
        
        # mask inlier-outlier from Huber:     
        #outlier_mask = huber.outliers_
        #inlier_mask = np.logical_not(outlier_mask)
    
        # Make predictions using the testing set
        # OLS:
        wind_pred = regr.predict(wt)  
        # Ransac:
        wind_ransac = ransac.predict(wt)
        # Huber:
        #wind_huber = huber.predict(wt)   
    
        fig, (ax1,ax2) = plt.subplots(ncols=2, sharey = True)
        # Plot outputs
        ax1.scatter(w_truth, w_opt,  c=group['Season'].map(colors), alpha= 0.5, label = 'Optimal vs True')
        ax1.plot(w_truth, wind_pred, color='black', linewidth=1.5, label = 'Linear regression')
        ax1.plot(w_truth, w_truth, color='gray', linewidth=1.5, label = 'Slope 1 line')

        ax1.set_ylabel('NS Wind Speed [m/s], Retrieved')
        ax1.set_xlabel('NS Wind Speed [m/s], True')
        ax1.grid(True)
        ax1.set_title('OLS '+windType)
        ax1.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
                '$mx+b$',
                np.float(regr.coef_),
                np.float(regr.intercept_),'m/s',
                rmse_local,'m/s',
                mae_local,'m/s',
                r2_local),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax1.transAxes)
        
        # Plot Ransac
        ax2.scatter(w_truth[inlier_mask], w_opt[inlier_mask],  
                c=group['Season'][inlier_mask].map(colors), alpha = 0.5, label = 'Retrieved')
        ax2.scatter(w_truth[outlier_mask], w_opt[outlier_mask],  
                color='magenta', alpha = 0.4, label = 'Outlier')
        ax2.plot(w_truth, wind_ransac, color='black', linewidth=1.5, label = 'Regression')
        ax2.plot(w_truth, w_truth, color='gray', linewidth=1.5, label = 'm = 1')
    
        ax2.legend()
        ax2.set_xlabel('NS Wind Speed [m/s], True')
        ax2.grid(True)
        ax2.set_title('Ransac '+windType)
        ax2.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
                '$mx+b$',
                np.float(ransac.estimator_.coef_),
                np.float(ransac.estimator_.intercept_),'m/s',
                np.sqrt(mean_squared_error(w_truth[inlier_mask], w_opt[inlier_mask])) ,'m/s',
                median_absolute_error(w_truth[inlier_mask], w_opt[inlier_mask]),'m/s',
                np.float(r2_score(w_truth[inlier_mask], w_opt[inlier_mask]))),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax2.transAxes)

        plt.legend(loc=(0.02,0.73), prop={'size': 9}) 
        plt.savefig('seas_wind'+name+windType+'_'+str(indice), dpi=300)  
        
        # Histogram
        bins_all = 5

        fig, ax = plt.subplots(1)
        ax.hist(error_local, bins = bins_all, label=name)
        ax.legend()
        ax.text(0.95,0.75,
                'Mean error = %.3g\nMedian error = %.3g\nBias (M) = %.3g' % ( 
                    mean_error_local, 
                    median_error_local,
                    multiplicativeBias),
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes)
    
        ax.grid(True)
        plt.savefig('seas_err'+name+windType+'_'+str(indice), dpi=300)
    
def regressPlots(truth, optimized, indice, windType = 'Wind'):
    
        #error_local = truth - optimized
        #mean_error_local = np.mean( error_local )
        #median_error_local = np.median( error_local )
    
        rmse_local = np.sqrt(mean_squared_error(truth, optimized)) 
        mae_local = median_absolute_error(truth, optimized)
        r2_local = r2_score(truth, optimized)

    
        # Ordinary least squares:
        regr = LinearRegression()
    
        # Robust fit with Ransac algo. (sklearn lib.)
        ransac = RANSACRegressor()
    
        # Train the model using the training sets
        wt = np.reshape(truth,(len(truth),1)) # for using with .fit below
    
        # OLS fit:
        regr.fit(wt, optimized)
        # Ransac fit:
        ransac.fit(wt, optimized)
        # mask inlier-outlier from Ransac:
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)
    
        # Make predictions using the testing set
        # OLS:
        wind_pred = regr.predict(wt)  
        # Ransac:
        wind_ransac = ransac.predict(wt)
    
        fig, (ax1,ax2) = plt.subplots(ncols=2, sharey = True)
        # Plot outputs
        ax1.scatter(truth, optimized,  color='dodgerblue', alpha = 0.5, label = 'Optimal vs True')
        ax1.plot(truth, wind_pred, color='black', linewidth=1.5, label = 'Linear regression')
        ax1.plot(truth, truth, color='gray', linewidth=1.5, label = 'Slope 1 line')

        ax1.set_ylabel('NS Wind Speed [m/s], Retrieved')
        ax1.set_xlabel('NS Wind Speed [m/s], True')
        ax1.grid(True)
        ax1.set_title('OLS '+windType)
        ax1.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
            '$mx+b$',
            np.float(regr.coef_),
            np.float(regr.intercept_),'m/s',
            rmse_local,'m/s',
            mae_local,'m/s',
            r2_local),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax1.transAxes)

        # Plot outputs
        ax2.scatter(truth[inlier_mask], optimized[inlier_mask],  
                color='dodgerblue', alpha = 0.5, label = 'Retrieved')
        ax2.scatter(truth[outlier_mask], optimized[outlier_mask],  
                color='magenta', alpha = 0.5, label = 'Outlier')
        ax2.plot(truth, wind_ransac, color='black', linewidth=1.5, label = 'Regression')
        ax2.plot(truth, truth, color='gray', linewidth=1.5, label = 'm = 1')
    
        ax2.legend()
        ax2.set_xlabel('NS Wind Speed [m/s], True')
        ax2.grid(True)
        ax2.set_title('Ransac '+windType)
        ax2.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
            '$mx+b$',
            np.float(ransac.estimator_.coef_),
            np.float(ransac.estimator_.intercept_),'m/s',
            np.sqrt(mean_squared_error(truth[inlier_mask], optimized[inlier_mask])) ,'m/s',
            median_absolute_error(truth[inlier_mask], optimized[inlier_mask]),'m/s',
            np.float(r2_score(truth[inlier_mask], optimized[inlier_mask]))),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax2.transAxes)

        plt.legend(loc=(0.02,0.73), prop={'size': 9}) 
        plt.savefig('windR_'+windType+str(indice), dpi=300)
        
        return rmse_local, mae_local, r2_local

